In [ ]:
'''
!!NOTE!!
!!User Input required!!

Parameters:

execution_mode (string): where the notebook is being run
    Sample: 'local', 'in-cluster'

host (string): KF Pipelines service endpoint
    Sample:  "http://10.10.10.10:31380/pipeline"

bucket_name (string): S3 bucket to be used by the pipeline
    Sample: "mxnet-model-store"

aws_secret_name (string): AWS secret where IAM creds are stored
    Sample: 'aws-secret'

role_arn (string): SageMaker Role ARN for execution of pipeline components
    Sample: 'arn:aws:iam::${account_id}:role/service-role/AmazonSageMaker-ExecutionRole-${timestemp}'
'''

execution_mode = 'in-cluster'
host = ''
bucket_name = 'mxnet-model-store'
aws_secret_name = 'aws-secret'
role_arn = ''

In [ ]:
if execution_mode == "local" and host == None:
    raise ValueError("Please set host to the appropriate URL")
elif execution_mode != "local":
    execution_mode = "in-cluster"

In [ ]:
import kfp
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client
from kfp.aws import use_aws_secret
import logging

In [ ]:
# Set loglevel to debug
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

model = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/aws/sagemaker/model/component.yaml'
deploy = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/aws/sagemaker/deploy/component.yaml'

In [ ]:
sagemaker_model_op = components.load_component_from_url(model)
sagemaker_deploy_op = components.load_component_from_url(deploy)

def blerssi_mxnet_train_upload_op(step_name='mxnet-train'):
    return dsl.ContainerOp(
        name='mxnet-train-upload-s3',
        image='ciscoai/mxnet-blerssi-train-upload:v0.2',
        command=['python', '/opt/mx-dnn.py', 'train'],
        arguments=['--bucket-name', bucket_name]
    ).apply(use_aws_secret(secret_name=aws_secret_name, aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))

In [ ]:
@dsl.pipeline(
    name='MXNet Sagemaker Hybrid Pipeline',
    description='Pipeline to train BLERSSI model using mxnet and save in aws s3 bucket'
)
def mxnet_pipeline(
    region="",
    image="",
    model_name="",
    endpoint_config_name="",
    endpoint_name="",
    model_artifact_url="",
    instance_type_1="",
    role=""
):
    train_upload_model = blerssi_mxnet_train_upload_op()

    create_model = sagemaker_model_op(
        region=region,
        model_name=model_name,
        image=image,
        model_artifact_url=model_artifact_url,
        role=role
    ).apply(use_aws_secret(secret_name=aws_secret_name, aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))
    create_model.after(train_upload_model)

    sagemaker_deploy_op(
        region=region,
        endpoint_config_name=endpoint_config_name,
        endpoint_name=endpoint_name,
        model_name_1=create_model.output,
        instance_type_1=instance_type_1
    ).apply(use_aws_secret(secret_name=aws_secret_name, aws_access_key_id_name='AWS_ACCESS_KEY_ID', aws_secret_access_key_name='AWS_SECRET_ACCESS_KEY'))

In [ ]:
try:
    import kfp.compiler as compiler
    compiler.Compiler().compile(mxnet_pipeline, 'mxnet_pipeline.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

In [ ]:
client = None
if execution_mode == "local":
    client = kfp.Client(host=host)
else:
    client = kfp.Client()
blerssi_hybrid_experiment = client.create_experiment(name='BLERSSI-Sagemaker')

## Run pipeline

Inference Image Source code: https://github.com/CiscoAI/cisco-kubeflow-starter-pack/tree/hybrid/apps/networking/ble-localization/hybrid-aws/pipelines/components/v1/mxnet-byom-inference/container

### Set pipeline parameters

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%y-%H-%M-%S")
logging.info("timestamp for pipeline artifacts: %s", timestamp)

# pipeline parameters :

# Region where the pipeline is supposed to push/pull artifacts
aws_region = 'us-west-2'

# pre-built inference image for serving the mxnet BLERSSI model
inference_image = '609615689023.dkr.ecr.us-west-2.amazonaws.com/mxnet-blerssi-inference:latest'

# model name to create a re-usable SageMaker Model resource
model_name = 'mxnet-blerssi-'+timestamp

# endpoint config name for the SageMaker Model Serving Endpoint Config
endpoint_config_name = 'mxnet-blerssi-endpoint-config-'+timestamp

# endpoint name for SageMaker Serving Endpoint
endpoint_name = 'mxnet-blerssi-'+timestamp

# model artifact URL
# Path to the model tarball 
model_path = 's3://'+bucket_name+'/blerssi/model.tar.gz'

# AWS instance type
instance_type = 'ml.m4.xlarge'

In [ ]:
run = client.run_pipeline(blerssi_hybrid_experiment.id, 'blerssi-sagemaker-pipeline-'+timestamp, pipeline_package_path='mxnet_pipeline.tar.gz', params={
    'region': aws_region,
    'image': inference_image,
    'model_name': model_name,
    'endpoint_config_name': endpoint_config_name,
    'endpoint_name': endpoint_name,
    'model_artifact_url': model_path,
    'instance_type_1': instance_type,
    'role': role_arn
})

## Model Validation with Endpoint

In [ ]:
import numpy
import os
import logging
import boto3
import json
import time

import pandas as pd
import numpy as np
import sagemaker
import mxnet as mx
from mxnet import nd
from sagemaker.mxnet import MXNetModel

In [ ]:
sagemaker_session = sagemaker.Session()
endpoint_status = 'Creating'

logging.info(f"Using endpoint: {endpoint_name}")
logging.info("Waiting for endpoint to be ready...")
time.sleep(120)

sg_client = boto3.client('sagemaker', region_name=aws_region)
while endpoint_status == 'Creating':
    resp = sg_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = resp['EndpointStatus']
    logging.info(f"Endpoint status: {endpoint_status}")
    if endpoint_status != 'Creating':
        break
    else:
        time.sleep(60)

logging.info(f"Endpoint {endpoint_name} is {endpoint_status}!")

In [ ]:
from sagemaker.predictor import json_serializer, json_deserializer, RealTimePredictor

predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker_session, content_type= 'application/x-npy', accept= 'application/json')

def _npy_dumps(data):
    """
    Serialized a numpy array into a stream of npy-formatted bytes.
    """
    from six import BytesIO
    buffer = BytesIO()
    np.save(buffer, data)
    return buffer.getvalue()

request_data = _npy_dumps(nd.array([[-200, -200, -200, -75, -200, -200, -200, -200, -200, -200, -200, -200, -200]]).asnumpy())
result = predictor.predict(data=request_data)

import pickle
depickled_result = pickle.loads(result)

print("Outputs, predictions")
print(depickled_result[0], depickled_result[1])

## (Optional) Delete endpoint

In [ ]:
logging.info("Deleting endpoint...")
predictor.delete_endpoint()